In [ ]:
import pandas as pd
import glob
import os

# 1~8 폴더명과 저장용 이름 매핑
folder_map = {
    "1.회원정보": "회원정보",
    "2.신용정보": "신용정보",
    "3.승인매출정보": "승인매출정보",
    "4.청구입금정보": "청구정보",
    "5.잔액정보": "잔액정보",
    "6.채널정보": "채널정보",
    "7.마케팅정보": "마케팅정보", 
    "8.성과정보": "성과정보"
}

base_dir = "../data/train/"  # 각자 상황에 맞게 수정

# 반복적으로 모든 데이터셋 처리
for folder, name in folder_map.items():
    path = os.path.join(base_dir, folder, "*.parquet")
    file_paths = sorted(glob.glob(path))
    
    if not file_paths:
        print(f"❌ {name}: 해당 폴더에 파일 없음 → {path}")
        continue
    
    print(f"📂 {name} 파일 수: {len(file_paths)}개")

    try:
        df_list = [pd.read_parquet(fp) for fp in file_paths]
        merged_df = pd.concat(df_list, ignore_index=True)
        
        print(f"✅ 병합 완료: {name} → shape: {merged_df.shape}")
        
        # 저장
        merged_df.to_parquet(f"{name}_train.parquet", index=False)
        print(f"💾 저장 완료: {name}_train.parquet")
    
    except Exception as e:
        print(f"❗ 오류 발생: {name} → {e}")


📂 회원정보 파일 수: 6개
✅ 병합 완료: 회원정보 → shape: (600000, 77)
💾 저장 완료: 회원정보_test.parquet
📂 신용정보 파일 수: 6개
✅ 병합 완료: 신용정보 → shape: (600000, 42)
💾 저장 완료: 신용정보_test.parquet
📂 승인매출정보 파일 수: 6개
✅ 병합 완료: 승인매출정보 → shape: (600000, 406)
💾 저장 완료: 승인매출정보_test.parquet
📂 청구정보 파일 수: 6개
✅ 병합 완료: 청구정보 → shape: (600000, 46)
💾 저장 완료: 청구정보_test.parquet
📂 잔액정보 파일 수: 6개
✅ 병합 완료: 잔액정보 → shape: (600000, 82)
💾 저장 완료: 잔액정보_test.parquet
📂 채널정보 파일 수: 6개
✅ 병합 완료: 채널정보 → shape: (600000, 105)
💾 저장 완료: 채널정보_test.parquet
📂 마케팅정보 파일 수: 6개
✅ 병합 완료: 마케팅정보 → shape: (600000, 64)
💾 저장 완료: 마케팅정보_test.parquet
📂 성과정보 파일 수: 6개
✅ 병합 완료: 성과정보 → shape: (600000, 49)
💾 저장 완료: 성과정보_test.parquet


In [ ]:
import pandas as pd
import os

# 기준 파일: 회원정보
merged = pd.read_parquet("회원정보_train.parquet")
print("✅ 기준: 회원정보 loaded", merged.shape)

# 기준년월 생성
if '기준년월' not in merged.columns:
    if '기준일자' in merged.columns:
        merged['기준년월'] = merged['기준일자'].astype(str).str[:6]
        print("🛠 기준 테이블: 기준년월 파생 완료")
    else:
        raise KeyError("❗ 기준 테이블에 '기준년월' 또는 '기준일자' 컬럼이 없습니다.")

# 병합할 다른 파일 목록
other_files = [
    "신용정보_train.parquet",
    "승인매출정보_train.parquet",
    "청구정보_train.parquet",
    "잔액정보_train.parquet",
    "채널정보_train.parquet",
    "마케팅정보_train.parquet",
    "성과정보_train.parquet"
]

for file in other_files:
    if os.path.exists(file):
        temp = pd.read_parquet(file)

        # 기준년월 생성
        if '기준년월' not in temp.columns:
            if '기준일자' in temp.columns:
                temp['기준년월'] = temp['기준일자'].astype(str).str[:6]
                print(f"🛠 {file}: 기준년월 파생 완료")
            else:
                print(f"⚠️ {file}: 기준년월 생성 불가 → 병합 건너뜀")
                continue

        # 중복 컬럼 제거 (ID, 기준년월 제외)
        overlap_cols = [col for col in temp.columns if col in merged.columns and col not in ['ID', '기준년월']]
        if overlap_cols:
            print(f"⚠️ 중복 컬럼 제거 ({file}):", overlap_cols)
            temp = temp.drop(columns=overlap_cols)

        merged = merged.merge(temp, on=['ID', '기준년월'], how='left')
        print(f"🔗 병합 완료: {file} → shape: {merged.shape}")
    else:
        print(f"❌ 파일 없음: {file}")

# 저장
merged.to_parquet("통합_train_데이터.parquet", index=False)
print("\n✅ 최종 통합 저장 완료: 통합_train_데이터.parquet")
print("📊 최종 shape:", merged.shape)


✅ 기준: 회원정보 loaded (600000, 77)
🔗 병합 완료: 신용정보_test.parquet → shape: (600000, 117)
🔗 병합 완료: 승인매출정보_test.parquet → shape: (600000, 521)
🔗 병합 완료: 청구정보_test.parquet → shape: (600000, 565)
🔗 병합 완료: 잔액정보_test.parquet → shape: (600000, 645)
🔗 병합 완료: 채널정보_test.parquet → shape: (600000, 748)
🔗 병합 완료: 마케팅정보_test.parquet → shape: (600000, 810)
🔗 병합 완료: 성과정보_test.parquet → shape: (600000, 857)

✅ 최종 통합 저장 완료: 통합_test_데이터.parquet
📊 최종 shape: (600000, 857)


In [5]:
import pandas as pd
import glob
import os

# 1~8 폴더명과 저장용 이름 매핑
folder_map = {
    "1.회원정보": "회원정보",
    "2.신용정보": "신용정보",
    "3.승인매출정보": "승인매출정보",
    "4.청구입금정보": "청구정보",
    "5.잔액정보": "잔액정보",
    "6.채널정보": "채널정보",
    "7.마케팅정보": "마케팅정보", 
    "8.성과정보": "성과정보"
}

base_dir = "../data/train/"  # 각자 상황에 맞게 수정

# 반복적으로 모든 데이터셋 처리
for folder, name in folder_map.items():
    path = os.path.join(base_dir, folder, "*.parquet")
    file_paths = sorted(glob.glob(path))
    
    if not file_paths:
        print(f"❌ {name}: 해당 폴더에 파일 없음 → {path}")
        continue
    
    print(f"📂 {name} 파일 수: {len(file_paths)}개")

    try:
        df_list = [pd.read_parquet(fp) for fp in file_paths]
        merged_df = pd.concat(df_list, ignore_index=True)
        
        print(f"✅ 병합 완료: {name} → shape: {merged_df.shape}")
        
        # 저장
        merged_df.to_parquet(f"{name}_train.parquet", index=False)
        print(f"💾 저장 완료: {name}_train.parquet")
    
    except Exception as e:
        print(f"❗ 오류 발생: {name} → {e}")


📂 회원정보 파일 수: 6개
✅ 병합 완료: 회원정보 → shape: (2400000, 78)
💾 저장 완료: 회원정보_train.parquet
📂 신용정보 파일 수: 6개
✅ 병합 완료: 신용정보 → shape: (2400000, 42)
💾 저장 완료: 신용정보_train.parquet
📂 승인매출정보 파일 수: 6개
✅ 병합 완료: 승인매출정보 → shape: (2400000, 406)
💾 저장 완료: 승인매출정보_train.parquet
📂 청구정보 파일 수: 6개
✅ 병합 완료: 청구정보 → shape: (2400000, 46)
💾 저장 완료: 청구정보_train.parquet
📂 잔액정보 파일 수: 6개
✅ 병합 완료: 잔액정보 → shape: (2400000, 82)
💾 저장 완료: 잔액정보_train.parquet
📂 채널정보 파일 수: 6개
✅ 병합 완료: 채널정보 → shape: (2400000, 105)
💾 저장 완료: 채널정보_train.parquet
📂 마케팅정보 파일 수: 6개
✅ 병합 완료: 마케팅정보 → shape: (2400000, 64)
💾 저장 완료: 마케팅정보_train.parquet
📂 성과정보 파일 수: 6개
✅ 병합 완료: 성과정보 → shape: (2400000, 49)
💾 저장 완료: 성과정보_train.parquet


In [6]:
import pandas as pd
import os

# 기준 파일: 회원정보
merged = pd.read_parquet("회원정보_train.parquet")
print("✅ 기준: 회원정보 loaded", merged.shape)

# 기준년월 생성
if '기준년월' not in merged.columns:
    if '기준일자' in merged.columns:
        merged['기준년월'] = merged['기준일자'].astype(str).str[:6]
        print("🛠 기준 테이블: 기준년월 파생 완료")
    else:
        raise KeyError("❗ 기준 테이블에 '기준년월' 또는 '기준일자' 컬럼이 없습니다.")

# 병합할 다른 파일 목록
other_files = [
    "신용정보_train.parquet",
    "승인매출정보_train.parquet",
    "청구정보_train.parquet",
    "잔액정보_train.parquet",
    "채널정보_train.parquet",
    "마케팅정보_train.parquet",
    "성과정보_train.parquet"
]

for file in other_files:
    if os.path.exists(file):
        temp = pd.read_parquet(file)

        # 기준년월 생성
        if '기준년월' not in temp.columns:
            if '기준일자' in temp.columns:
                temp['기준년월'] = temp['기준일자'].astype(str).str[:6]
                print(f"🛠 {file}: 기준년월 파생 완료")
            else:
                print(f"⚠️ {file}: 기준년월 생성 불가 → 병합 건너뜀")
                continue

        # 중복 컬럼 제거 (ID, 기준년월 제외)
        overlap_cols = [col for col in temp.columns if col in merged.columns and col not in ['ID', '기준년월']]
        if overlap_cols:
            print(f"⚠️ 중복 컬럼 제거 ({file}):", overlap_cols)
            temp = temp.drop(columns=overlap_cols)

        merged = merged.merge(temp, on=['ID', '기준년월'], how='left')
        print(f"🔗 병합 완료: {file} → shape: {merged.shape}")
    else:
        print(f"❌ 파일 없음: {file}")

# 저장
merged.to_parquet("통합_train_데이터.parquet", index=False)
print("\n✅ 최종 통합 저장 완료: 통합_train_데이터.parquet")
print("📊 최종 shape:", merged.shape)


✅ 기준: 회원정보 loaded (2400000, 78)
🔗 병합 완료: 신용정보_train.parquet → shape: (2400000, 118)
🔗 병합 완료: 승인매출정보_train.parquet → shape: (2400000, 522)
🔗 병합 완료: 청구정보_train.parquet → shape: (2400000, 566)
🔗 병합 완료: 잔액정보_train.parquet → shape: (2400000, 646)
🔗 병합 완료: 채널정보_train.parquet → shape: (2400000, 749)
🔗 병합 완료: 마케팅정보_train.parquet → shape: (2400000, 811)
🔗 병합 완료: 성과정보_train.parquet → shape: (2400000, 858)

✅ 최종 통합 저장 완료: 통합_train_데이터.parquet
📊 최종 shape: (2400000, 858)
